In [1]:
'''
!pip install datasets

from datasets import load_dataset
dataset = load_dataset(
   'web_nlg', 'webnlg_challenge_2017')

!pip install transformers 
pip install sentencepiece

!pip install sacrebleu
'''

"\n!pip install datasets\n\nfrom datasets import load_dataset\ndataset = load_dataset(\n   'web_nlg', 'webnlg_challenge_2017')\n\n!pip install transformers \npip install sentencepiece\n\n!pip install sacrebleu\n"

In [2]:
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset

# define the tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-small')
model = T5ForConditionalGeneration.from_pretrained('t5-small')


In [3]:
new_tokens = ['<H>', '<R>', '<T>']
new_tokens_vocab = {}
new_tokens_vocab['additional_special_tokens'] = []
for idx, t in enumerate(new_tokens):
    new_tokens_vocab['additional_special_tokens'].append(t)
num_added_toks = tokenizer.add_special_tokens(new_tokens_vocab)

tokenizer.add_tokens("[MASK]")
tokenizer.mask_token = "[MASK]"
tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

In [4]:
class WebNLGDataset(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        # preprocess the input graph
        try:
            triples = item['original_triple_sets']['otriple_set']
            input_text = self.prefix
            for outer_list in triples: 
                for triple in outer_list:
                    triple_txt = triple.split("|")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
        except (KeyError, IndexError):
            print("1")
            print(item['original_triple_sets']['otriple_set'])
            print(item['original_triple_sets']['otriple_set'][0])
            print(triples)
            input_text = self.prefix
        # preprocess the target text
        try:
            target_text = item['lex']['text'][0]
        except (KeyError, IndexError):
            print("2")
            print(item)
            #print(item['original_triple_sets']['otriple_set'])
            target_text = ""
        #print(item)
        #print(input_text)
        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [5]:
MAX_INPUT_LENGTH = 128
MAX_TARGET_LENGTH = 128
tokenizer.model_max_length = MAX_INPUT_LENGTH
model.config.max_length = MAX_TARGET_LENGTH

# set up the device (GPU or CPU)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [6]:
# load the WebNLG dataset
dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['train']

Generating train split:   0%|          | 0/6940 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4615 [00:00<?, ? examples/s]

Dataset web_nlg downloaded and prepared to /root/.cache/huggingface/datasets/web_nlg/webnlg_challenge_2017/0.0.0/28ffb892f7f42450dd9558684aa43bcaf44b1b3bf0d77cb8d73534646af88dda. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Adaptive pretraining

For STA, we fine-tuned the PLMs on a small amount of labeled data from the target task using a maximum likelihood estimation (MLE) objective. This involves training the model to maximize the likelihood of generating the correct output given the input graph and labeled data. This process helps to further adapt the PLM to the specific requirements of the target task and improve its performance on that task.

In [ ]:
import random

pretrain_texts = []
for sample in dataset:
    try:
        text = sample['lex']['text'][0]
        pretrain_texts.append(text)
    except (KeyError, IndexError):
        continue

tokenized_inputs = tokenizer(pretrain_texts, truncation=True, padding=True, max_length=128, return_tensors='pt')
input_ids = tokenized_inputs['input_ids']
attention_mask = tokenized_inputs['attention_mask']

pretrain_data = torch.utils.data.TensorDataset(input_ids, attention_mask)

pretrain_loader = torch.utils.data.DataLoader(pretrain_data, batch_size=int(60), shuffle=True)

pretrain_optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
pretrain_criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

pretrain_epochs = 3  # Set the number of pre-training epochs
masking_prob = 0.15  # Probability of masking a token

if tokenizer.mask_token is None:
    # Manually set a mask token if not already defined
    tokenizer.add_tokens("[MASK]")
    tokenizer.mask_token = "[MASK]"
    tokenizer.mask_token_id = tokenizer.convert_tokens_to_ids("[MASK]")

for epoch in range(pretrain_epochs):
    running_loss = 0.0
    for inputs, attention_mask in pretrain_loader:
        inputs = inputs.to(device)
        attention_mask = attention_mask.to(device)
        batch_size, seq_length = inputs.shape
        
        # Create a mask for randomly selected tokens
        mask = torch.rand(inputs.shape) < masking_prob
        
        # Randomly replace selected tokens with [MASK] token
        masked_inputs = inputs.clone()
        masked_inputs[mask] = tokenizer.mask_token_id
        
        pretrain_optimizer.zero_grad()
        outputs = model(input_ids=masked_inputs, attention_mask=attention_mask, decoder_input_ids=inputs)
        
        # Compute the loss only for the masked tokens
        masked_logits = outputs.logits[mask]
        masked_labels = inputs[mask]
        loss = pretrain_criterion(masked_logits.view(-1, masked_logits.size(-1)), masked_labels.view(-1))
        
        loss.backward()
        pretrain_optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    
    epoch_loss = running_loss / len(pretrain_data)
    print(f"Pretrain Epoch {epoch+1}/{pretrain_epochs} - loss: {epoch_loss:.4f}")


For LMA, we first fine-tuned the PLMs on a small amount of task-specific data using a masked language modeling objective. This involves randomly masking some tokens in the input sequence and training the model to predict the masked tokens based on the context provided by the unmasked tokens. This process helps to adapt the PLM to the specific characteristics of the target task and improve its performance on that task.

# Finetuning

In [11]:
# set up the data loader
train_data = WebNLGDataset(dataset)
batch_size = 32 #16
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

In [ ]:
# set up the optimizer and the loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4) #3e-5
criterion = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)


In [ ]:
# fine-tune the model
num_epochs = 3
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, targets in train_loader:
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs, labels=targets)
        loss = criterion(outputs.logits.view(-1, outputs.logits.size(-1)), targets.view(-1))
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    epoch_loss = running_loss / len(train_data)
    print(f"Epoch {epoch+1}/{num_epochs} - loss: {epoch_loss:.4f}")


In [7]:
# Save the entire model
torch.save(model, 'model_with_CCE_masked_pretraining_multipe_triples_71')
print("Model saved successfully.")

Model saved successfully.


In [8]:
# Load the model
model = torch.load('/kaggle/input/models/model_with_CCE_masked_pretraining_multipe_triples_71')

# Print a confirmation message
print("Model loaded successfully.")

Model loaded successfully.


## are we accounting for the multiple texts targets in the bleu? it doesn't look like it

In [9]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 4.4 MB/s eta 0:00:00


In [12]:
from sacrebleu import corpus_bleu
from random import sample
from tqdm import tqdm

# load the WebNLG validation dataset
validation_dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['test']
validation_dataset = [sample for sample in validation_dataset if sample['lex']['text']] # filter out samples with empty targets 
# Select a subset of the validation dataset
#subset_size = 10  # Choose the desired subset size
#validation_subset = sample(list(validation_dataset), subset_size)
validation_data = WebNLGDataset(validation_dataset)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model.eval()

# generate predictions for the validation dataset
predictions = []
references = []
with torch.no_grad():
    for inputs, targets in tqdm(validation_loader, desc='Validation Progress', leave=False):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions.extend(predicted_texts)
        references.extend(target_texts)

# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

references = []
for i in range(len(validation_dataset)):
    references.append(validation_dataset[i]['lex']['text'])
bleu = corpus_bleu(predictions, references)

print(f"BLEU score: {bleu.score}")


  0%|          | 0/3 [00:00<?, ?it/s]

BLEU score: 71.86082239261687


-----

In [14]:
!pip install xmltodict

In [23]:
import xmltodict

def load_xml_as_dict(file_path):
    with open(file_path, 'r') as xml_file:
        xml_data = xml_file.read()
        data_dict = xmltodict.parse(xml_data, force_list=('entry',))
        return data_dict['benchmark']['entries']['entry']


In [24]:
unseen = load_xml_as_dict('/kaggle/input/webnlg/testdata_unseen_with_lex.xml')

In [80]:
unseen[50]['originaltripleset']
input_text =''
triples = unseen[50]['originaltripleset']
for triple in triples:
                    triple_txt = triple['otriple'].split(" | ")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]

In [82]:
unseen[16]

{'@category': 'Politician',
 '@eid': 'Id17',
 '@size': '1',
 'originaltripleset': {'otriple': 'Adonis_Georgiadis | office | "Deputy Minister for Development, Competitiveness and Shipping"'},
 'modifiedtripleset': {'mtriple': 'Adonis_Georgiadis | office (workedAt, workedAs) | "Deputy Minister for Development, Competitiveness and Shipping"'},
 'lex': [{'@comment': 'good',
   '@lid': 'Id1',
   '#text': 'Adonis Georgiadis worked as Deputy Minister for Development, Competitiveness and Shipping.'},
  {'@comment': 'good',
   '@lid': 'Id2',
   '#text': 'Adonis Georgiadis was the Deputy Minister for Development, Competitiveness and Shipping.'},
  {'@comment': 'good',
   '@lid': 'Id3',
   '#text': 'Adonis Georgiadis worked as the Deputy Minister for Development, Competitiveness and Shipping.'}]}

In [65]:
unseen[30]['originaltripleset']['otriple']

'Alfred_Moore_Scales | activeYearsStartDate | "1875-03-04"^^xsd:date'

In [73]:
class WebNLGDataset_unseen(Dataset):
    def __init__(self, data):
        self.data = data
        self.prefix = "translate from Graph to Text: "

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        
        # preprocess the input graph
        try:
            triples = item['originaltripleset']['otriple']
            input_text = self.prefix
            if isinstance(triples, str):
                triple_txt = triples.split(" | ")
                input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
            elif isinstance(triples, list):
                for triple in triples:
                    triple_txt = triple.split(" | ")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
        except:
            try:
                triples = item['originaltripleset']
                for triple in triples:
                    triple_txt = triple['otriple'].split(" | ")
                    input_text += " <H> " + triple_txt[0] + " <R> " + triple_txt[1] + " <T> " + triple_txt[2]
                
            except:
                print("Error: Unable to process input graph.")
                print(item)
                input_text = self.prefix

        # preprocess the target text
        try:
            target_text = item['lex'][0]['#text']
        except (KeyError, IndexError):
            try:
                target_text = item['lex']['#text']
            except:
                print("Error: Unable to process target text.")
                target_text = ""

        # encode the inputs and targets using the tokenizer
        input_ids = tokenizer.encode(input_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        target_ids = tokenizer.encode(target_text, return_tensors='pt', padding='max_length', max_length=128, truncation=True)
        
        return input_ids.squeeze(0), target_ids.squeeze(0)


In [74]:
validation_data = WebNLGDataset_unseen(unseen)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model.eval()

# generate predictions for the validation dataset
predictions = []
references = []
i=0
with torch.no_grad():
    for inputs, targets in tqdm(validation_loader, desc='Validation Progress', leave=False):
        i = i + 1
        print(i)

Validation Progress:  18%|█▊        | 5/28 [00:00<00:00, 46.47it/s]

Error: Unable to process input graph.
{'@category': 'Politician', '@eid': 'Id16', '@size': '1', 'originaltripleset': [{'otriple': 'Adonis_Georgiadis | primeminister | Antonis_Samaras'}, {'otriple': 'Adonis_Georgiadis | primeMinister | Antonis_Samaras'}], 'modifiedtripleset': {'mtriple': 'Adonis_Georgiadis | inOfficeWhilePrimeMinister | Antonis_Samaras'}, 'lex': [{'@comment': 'good', '@lid': 'Id1', '#text': 'Adonis Georgiadis was in office while Antonis Samaras was Prime Minister.'}, {'@comment': 'good', '@lid': 'Id2', '#text': 'Adonis Georgiadis was in office under Antonis Samaras, Prime Minister.'}, {'@comment': 'good', '@lid': 'Id3', '#text': 'Adonis Georgiadis served in office while Antonis Samaras was Prime Minister.'}]}
Error: Unable to process input graph.
{'@category': 'Politician', '@eid': 'Id19', '@size': '1', 'originaltripleset': [{'otriple': 'Agnes_Ward_White | placeOfBirth | Marietta,_Ohio'}, {'otriple': 'Agnes_Ward_White | birthPlace | Marietta,_Ohio'}], 'modifiedtripleset

Validation Progress:  54%|█████▎    | 15/28 [00:00<00:00, 40.77it/s]

Error: Unable to process input graph.
{'@category': 'Athlete', '@eid': 'Id307', '@size': '2', 'originaltripleset': [{'otriple': ['RoPS | league | Veikkausliiga', 'Akeem_Priestley | clubs | RoPS']}, {'otriple': ['RoPS | league | Veikkausliiga', 'Akeem_Priestley | team | RoPS']}], 'modifiedtripleset': {'mtriple': ['RoPS | league | Veikkausliiga', 'Akeem_Priestley | club | RoPS']}, 'lex': [{'@comment': 'good', '@lid': 'Id1', '#text': 'Akeem Priestley’s club is RoPS which plays in the Veikkausliiga.'}, {'@comment': 'good', '@lid': 'Id2', '#text': 'Akeem Priestley plays for the club RoPS in the Veikkausliiga league.'}, {'@comment': 'good', '@lid': 'Id3', '#text': 'Akeem Priestley played for club RoPS in the Veikkausliiga.'}]}
Error: Unable to process input graph.
{'@category': 'Athlete', '@eid': 'Id308', '@size': '2', 'originaltripleset': [{'otriple': ['United_Petrotrin_F.C. | ground | Palo_Seco_Velodrome', 'Akeem_Adams | clubs | United_Petrotrin_F.C.']}, {'otriple': ['United_Petrotrin_F.C.

Validation Progress:  71%|███████▏  | 20/28 [00:00<00:00, 36.10it/s]

Error: Unable to process input graph.
{'@category': 'MeanOfTransportation', '@eid': 'Id548', '@size': '3', 'originaltripleset': [{'otriple': ['Atlas_II | sites | Cape_Canaveral_Air_Force_Station', 'Atlas_II | countryOrigin | United_States', 'Atlas_II | launchSite | Spaceport_Florida_Launch_Complex_36']}, {'otriple': ['Atlas_II | countryOrigin | United_States', 'Atlas_II | launchSite | Cape_Canaveral_Air_Force_Station', 'Atlas_II | sites | Spaceport_Florida_Launch_Complex_36']}], 'modifiedtripleset': {'mtriple': ['Atlas_II | launchSite | Cape_Canaveral_Air_Force_Station', 'Atlas_II | countryOrigin | United_States', 'Atlas_II | launchSite | Spaceport_Florida_Launch_Complex_36']}, 'lex': [{'@comment': 'good', '@lid': 'Id1', '#text': 'Atlas II originates from the United States, and was launched from the Cape Canaveral Air Force Station in Spaceport Florida Launch Complex 36.'}, {'@comment': 'good', '@lid': 'Id2', '#text': 'Originating from the U.S, the Atlas II was launched from the Spacep

Error: Unable to process input graph.
{'@category': 'Politician', '@eid': 'Id743', '@size': '4', 'originaltripleset': [{'otriple': ['Adonis_Georgiadis | birthPlace | Athens', 'Adonis_Georgiadis | placeOfBirth | Greece', 'Adonis_Georgiadis | office | "Vice-President of New Democracy"@en', 'Adonis_Georgiadis | successor | Makis_Voridis']}, {'otriple': ['Adonis_Georgiadis | birthPlace | Greece', 'Adonis_Georgiadis | office | "Vice-President of New Democracy"@en', 'Adonis_Georgiadis | placeOfBirth | Athens', 'Adonis_Georgiadis | successor | Makis_Voridis']}], 'modifiedtripleset': {'mtriple': ['Adonis_Georgiadis | birthPlace | Athens', 'Adonis_Georgiadis | birthPlace | Greece', 'Adonis_Georgiadis | office (workedAt, workedAs) | "Vice-President of New Democracy"', 'Adonis_Georgiadis | successor | Makis_Voridis']}, 'lex': [{'@comment': 'good', '@lid': 'Id1', '#text': 'Adonis Georgiadis was born in Athens, Greece. He was Vice-president of new democracy and was succeeded by Makis Voridis.'}, {'

In [ ]:
validation_data = WebNLGDataset_unseen(unseen)

# set up the validation data loader
validation_loader = DataLoader(validation_data, batch_size=batch_size, shuffle=False)

# switch model to evaluation mode
model.eval()

# generate predictions for the validation dataset
predictions = []
references = []
with torch.no_grad():
    for inputs, targets in tqdm(validation_loader, desc='Validation Progress', leave=False):
        inputs = inputs.to(device)
        targets = targets.to(device)
        outputs = model.generate(inputs, max_length=MAX_TARGET_LENGTH, num_beams=4)
        # convert token IDs to strings
        predicted_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        target_texts = tokenizer.batch_decode(targets, skip_special_tokens=True)
        # append predicted and target texts for BLEU evaluation
        predictions.extend(predicted_texts)
        references.extend(target_texts)

# calculate BLEU scores
#bleu = corpus_bleu(predictions, [references])

references = []
for i in range(len(validation_dataset)):
    references.append(validation_dataset[i]['lex']['text'])
bleu = corpus_bleu(predictions, references)

print(f"BLEU score: {bleu.score}")

In [ ]:
i=5
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(references[i])

In [ ]:
i=10
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(references[i])

In [ ]:
i=50
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(references[i])

In [ ]:
i=0
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(references[i])

### known error, sometimes the prompt leaks into the output 

In [ ]:
i=70
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(references[i])

In [ ]:
i=130
print(validation_dataset[i])
print(validation_dataset[i]['original_triple_sets']['otriple_set'])
print(predictions[i])
print(references[i])

## there is a problem with empty target samples in the test set, we still need to check multiple triples!

In [ ]:
from transformers import T5Tokenizer
from datasets import load_dataset

# Define the tokenizer
#tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the WebNLG dataset
dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['test']
dataset = [sample for sample in dataset if sample['lex']['text']]

# Create an instance of WebNLGDataset
webnlg_dataset = WebNLGDataset(dataset)

# Define the index of the example you want to test
example_index = 70

# Get the input and target texts for the example at the specified index
input_text, target_text = webnlg_dataset[example_index]

# Decode the input and target texts using the tokenizer
decoded_input_text = tokenizer.decode(input_text, skip_special_tokens=True)
decoded_target_text = tokenizer.decode(target_text, skip_special_tokens=True)

# Print the preprocessed input and target texts
print("Input Text:", decoded_input_text)
print("Target Text:", decoded_target_text)


## seeing how many empty targets there are in the testing set

In [ ]:
dataset = load_dataset('web_nlg', 'webnlg_challenge_2017')['test']
count_empty_text = 0
for sample in dataset:
    if not sample['lex']['text']:
        count_empty_text += 1

print(f"Number of samples with empty 'lex' 'text' field: {count_empty_text}")


In [ ]:
total_samples = len(dataset)
print(f"Total number of samples in the test dataset: {total_samples}")